# DS Automation Assignment

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

*Optional* challenges:
- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's `input()` function, the `click` package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1.

In [1]:
import pandas as pd
df = pd.read_csv('/home/plato/Documents/Regis/Week 5 - Automation and Data Science/prepped_churn_data_new.csv', index_col='customerID')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn
customerID,,,,,,,
0,1,0,0.083333,1,29.85,29.85,0
1,34,1,1.000000,2,56.95,1889.50,0
2,2,1,0.083333,2,53.85,108.15,1
3,45,0,1.000000,3,42.30,1840.75,0
4,2,1,0.083333,1,70.70,151.65,1
...,...,...,...,...,...,...,...
7038,24,1,1.000000,2,84.80,1990.50,0
7039,72,1,1.000000,4,103.20,7362.90,0
7040,11,0,0.083333,1,29.60,346.45,0


# Use pycaret to find an ML algorithm that performs best on the data
Following the PEP8 guide, we import the ClassificationExperiment class straight from the pycaret.classification module and initialize the class.

In [2]:
from pycaret.classification import ClassificationExperiment

In [3]:
automl = ClassificationExperiment()

Then we call the function 'setup' which initializes the training environment and creates the transformation pipeline.

In [4]:
automl.setup(df, target='Churn')

,Description,Value
0,Session id,2060
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7043, 7)"
4,Transformed data shape,"(7043, 7)"
5,Transformed train set shape,"(4930, 7)"
6,Transformed test set shape,"(2113, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


In [5]:
automl

# Choose a metric you think is best to use for finding the best model
Here, we chose the AUC method rather than the default accuracy metric because it found the model with the highest precision.

In [6]:
best_model = automl.compare_models(sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7901,0.8349,0.4862,0.6375,0.5510,0.4174,0.4243,0.5220
lr,Logistic Regression,0.7919,0.8326,0.5092,0.6347,0.5645,0.4300,0.4349,0.6250
ada,Ada Boost Classifier,0.7860,0.8303,0.4832,0.6250,0.5442,0.4076,0.4138,0.2950
lightgbm,Light Gradient Boosting Machine,0.7866,0.8249,0.5008,0.6235,0.5547,0.4167,0.4215,0.4390
qda,Quadratic Discriminant Analysis,0.7465,0.8224,0.7416,0.5164,0.6086,0.4301,0.4456,0.0600
lda,Linear Discriminant Analysis,0.7884,0.8181,0.4916,0.6300,0.5518,0.4162,0.4219,0.0480
nb,Naive Bayes,0.7168,0.8040,0.7554,0.4790,0.5861,0.3870,0.4104,0.0560
rf,Random Forest Classifier,0.7716,0.7946,0.4778,0.5856,0.5256,0.3774,0.3811,0.5820
et,Extra Trees Classifier,0.7635,0.7760,0.4870,0.5645,0.5219,0.3661,0.3685,0.4860
knn,K Neighbors Classifier,0.7538,0.7386,0.4220,0.5492,0.4763,0.3191,0.3244,0.0700


the best model is Gradient Boosting Classifier with AUC score of 0.8349. The reason the model works best for our data is because it provides high predictive accuracy by combining weak learners in an ensemble to create a strong predictor.

In [7]:
best_model

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=2060, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

# Save the model to disk
We save the model as a pkl file. We used pickle to serialize our pycaret model into a binary format and deserialize it back to the python object.

In [8]:
automl.save_model(best_model, 'pycaret_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'PhoneService',
                                              'Contract', 'PaymentMethod',
                                              'MonthlyCharges', 'TotalCharges'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',...
                                             criterion='friedman_mse', init=None,
                      

# Check our saved model

We then load our saved model again and use it to predict the churn probability of our second to last row of the churn dataset.

In [9]:
new_pycaret = ClassificationExperiment()
loaded_model = new_pycaret.load_model('pycaret_model')

Transformation Pipeline and Model Successfully Loaded


In [10]:
new_pycaret.predict_model(loaded_model, df.iloc[-2:-1])

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,prediction_label,prediction_score
customerID,,,,,,,,,
7041,4,1,0.083333,2,74.400002,306.600006,1,1,0.5825


# Create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
We create a file called 'predict_churn_pycaret.py' that predicts the churn probability of each row in any dataset similar to our prepared dataset. Our file contains 2 functions that loads the new dataset and predicts the churn probability of that dataset. We modified the names in the 'prediction_label' column to make it easier for understanding the prediction.  

In [15]:
from IPython.display import Code

Code('predict_churn_pycaret.py')

import pandas as pd
from pycaret.classification import ClassificationExperiment

filepath = '/home/plato/Documents/Regis/Week 5 - Automation and Data Science/new_churn_data.csv'

def load_data(filepath):
    # Loading our new churn data to be predicted
    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    # making churn predictions using the loaded data
    classifier = ClassificationExperiment()
    # retrieving our previously trained gbc model
    model = classifier.load_model('pycaret_model')           
    predictions = classifier.predict_model(model, data=df)
    # changing the default prediction_label column in predict_model method to our preferred column name and value name 
    predictions.rename({'prediction_label': 'Churn'}, axis=1, inplace=True)
    predictions['Churn'].replace({1: 'Churn', 0: 'No Churn'},
                                            inplace=True)
   
    return predictions


if __name__ == "__main__":
    df = load_data(filepath)
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

# Testing the Python module and function with the new data, new_churn_data.csv
Here, we can see the churn probability of the new data. Comparing the performance of our model with the true values for the new data, our model predicted 4 out of 5 correctly which is considerably good. However, if we look at each prediction probality, our model predicted the first correct answer with the probability score of 0.55 while also predicting the wrong answer with a probability score of 0.72. The reason for these mistakes is probably because of insufficient data similar to these 2 rows.  

In [13]:
%run predict_churn_pycaret.py

Transformation Pipeline and Model Successfully Loaded
predictions:
            tenure  PhoneService  Contract  PaymentMethod  MonthlyCharges  \
customerID                                                                  
9305-CKSKC      22             1         0              2       97.400002   
1452-KNGVK       8             0         1              1       77.300003   
6723-OKKJM      28             1         0              0       28.250000   
7832-POPKP      62             1         0              2      101.699997   
6348-TACGU      10             0         0              1       51.150002   

            TotalCharges  charge_per_tenure     Churn  prediction_score  
customerID                                                               
9305-CKSKC    811.700012          36.895454     Churn            0.5572  
1452-KNGVK   1701.949951         212.743744  No Churn            0.8458  
6723-OKKJM    250.899994           8.960714  No Churn            0.8755  
7832-POPKP   3106.56005

# Summary

For this week's assignment, we used pycaret to find an ML algorithm that performs best on the data using AUC metric. We chose gradient boosting classifier as our model because it performed the best with the score of 0.82 than any other metric score or model. Then we saved the model to disk as a pkl file in order to use it again with new data. After that, we created a Python file with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in a new dataframe. Finally, we tested our Python file and function with the new data, new_churn_data.csv. Our model correctly predicted 4/5 of the new churn dataset. 